0. Importing libraries/Defining dataframes

In [35]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [36]:
# The datasets that will be used
data = pd.read_csv("learningSet_ml.csv", index_col=[0])
numerical = pd.read_csv('numerical.csv', index_col=[0])

In [37]:
# Column name standardisation
# for learning set
cols = [col_name.lower().replace(' ', '_') for col_name in data.columns]
data.columns = cols
# for numerical
cols = [col_name.lower().replace(' ', '_') for col_name in numerical.columns]
numerical.columns = cols

## Lab | Feature Engineering

#### 1. Checking for nulls in the numerical columns.

In [38]:
numerical

,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,stategov,...,cardgift,minramnt,maxramnt,lastgift,timelag,avggift,controln,hphone_d,rfa_2f,cluster2
TCODE,,,,,,,,,,,,,,,,,,,,,
0,60.000000,5,9,0,0,39,34,18,10,2,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,46.000000,6,9,16,0,15,55,11,6,2,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
1,61.611649,3,1,2,0,20,29,33,6,8,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
0,70.000000,1,4,2,0,23,14,31,3,0,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
0,78.000000,3,2,60,1,28,9,53,26,3,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,61.611649,5,9,0,14,36,47,11,7,8,...,0,25.0,25.0,25.0,9,25.000000,184568,0,1,12
1,48.000000,7,9,1,0,31,43,19,4,1,...,0,20.0,20.0,20.0,9,20.000000,122706,1,1,2
1,60.000000,5,9,0,0,18,46,20,7,23,...,4,3.0,10.0,10.0,3,8.285714,189641,1,3,34


In [39]:
# Checking for nulls
nulls_num = numerical.isna().sum()*100/len(numerical)
nulls_num_df = pd.DataFrame(nulls_num, columns = ['null_percentage']).reset_index().rename(columns = {'index' : 'column_names'})
print("Number of NaNs in numerical columns:""\n", nulls_num_df.sort_values(by = 'null_percentage', ascending = False))

Number of NaNs in numerical columns:
     column_names  null_percentage
0            age              0.0
206         eic4              0.0
213        eic11              0.0
212        eic10              0.0
211         eic9              0.0
..           ...              ...
104        ethc2              0.0
103        ethc1              0.0
102        hhd12              0.0
101        hhd11              0.0
313     cluster2              0.0

[314 rows x 2 columns]


#### *There are no nulls anywhere in the numerical subset*

#### 2. Cleaning columns GEOCODE2, WEALTH1, ADI, DMA,and MSA
<br>*First a closer look will into each feature's characteristics is needed in order to make a decision on how to clean each column*

- geocode2

In [40]:
data.geocode2.unique()

array(['C', 'A', 'D', 'B', ' ', nan], dtype=object)

In [41]:
data.geocode2.value_counts(dropna = False)

A      34484
B      28505
D      16580
C      15524
         187
NaN      132
Name: geocode2, dtype: int64

In [42]:
data.geocode2.describe()

count     95280
unique        5
top           A
freq      34484
Name: geocode2, dtype: object

As shown above, geocode2 has 132 NaNs and 187 white spaces (a total of 319 data points to handle), 95,280 datapoints in total and the mode (A) frequency is 34,484. In this case, changing those 319 values to the mode value would have practically no effect on the variable's distribution. However, there are rather considerable imbalances in the data.

In [43]:
# Function to clean geocode2
def clean_geocode2(x):
    if x == " ":
        return "A"
    else:
        return x

# Applying function to the column and replacing nans with the mode
data['geocode2'] = data['geocode2'].apply(lambda x : clean_geocode2(x)).fillna("A")

In [44]:
data.geocode2.value_counts(dropna = False)

A    34803
B    28505
D    16580
C    15524
Name: geocode2, dtype: int64

- wealth1

In [45]:
data.wealth1.unique()

array([nan,  9.,  1.,  4.,  2.,  6.,  0.,  5.,  8.,  3.,  7.])

In [46]:
data.wealth1.value_counts(dropna = False)

NaN    44732
9.0     7585
8.0     6793
7.0     6198
6.0     5825
5.0     5280
4.0     4810
3.0     4237
2.0     4085
1.0     3454
0.0     2413
Name: wealth1, dtype: int64

In [47]:
data.wealth1.describe()

count    50680.000000
mean         5.345699
std          2.742490
min          0.000000
25%          3.000000
50%          6.000000
75%          8.000000
max          9.000000
Name: wealth1, dtype: float64

With 44,732 NaNs, wealth1 needs to be handled in a more comprehensive manner and thus the KNNImputer will be used to fill in the missing values.

In [48]:
# Attempting to predict the missing values with XGBoost
# categorical features need to be turned to type 'category'
wealth1_notna = data[data['wealth1'].notna()]
for column in wealth1_notna.columns:
    if wealth1_notna[column].dtype == object or column == 'wealth1':
        wealth1_notna[column] = wealth1_notna[column].astype('category')

In [49]:
# X-y and train/test splits
X = wealth1_notna.drop('wealth1', axis = 1)
y = wealth1_notna['wealth1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

In [50]:
# Running the model
model = XGBRegressor(n_estimators = 400, booster = 'gbtree', objective = 'reg:squarederror', eta = 0.1, subsample = 0.6, tree_method = 'hist', num_parallel_tree = 3, enable_categorical = True)

eval_set = [(X_train, y_train), (X_test, y_test) ]
model.fit(X_train, y_train, eval_metric = "rmse", eval_set = eval_set, verbose = 20, early_stopping_rounds=10)

# Getting predictions
y_pred = model.predict(X_test)
# Getting scores
r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred, squared=False)
MAE = mean_absolute_error(y_test, y_pred)

print("r2 =", round(r2,2))
print("MSE =", round(MSE,2))
print("RMSE =", round(RMSE,2))
print("MAE =", round(MAE,2))

[0]	validation_0-rmse:5.03657	validation_1-rmse:5.06843
[20]	validation_0-rmse:1.02224	validation_1-rmse:1.32274
[40]	validation_0-rmse:0.67269	validation_1-rmse:1.13326
[60]	validation_0-rmse:0.61914	validation_1-rmse:1.12402
[80]	validation_0-rmse:0.58441	validation_1-rmse:1.12235
[100]	validation_0-rmse:0.55521	validation_1-rmse:1.12119
[120]	validation_0-rmse:0.52986	validation_1-rmse:1.12065
[140]	validation_0-rmse:0.50720	validation_1-rmse:1.12016
[160]	validation_0-rmse:0.48601	validation_1-rmse:1.11983
[180]	validation_0-rmse:0.46764	validation_1-rmse:1.11977
[182]	validation_0-rmse:0.46592	validation_1-rmse:1.11978
r2 = 0.84
MSE = 1.25
RMSE = 1.12
MAE = 0.71


In [51]:
# Here, xgboost was initially used to predict the missing values. However, as the dataset has not been properly processed, hyperparameter tuning couldn't make up for the complexity and chaotic nature of this big dataset and the predictions made weren't precise enough.
# A great alternative to that is the method used by @ieumann (Github), where the missing values were imputed proportionately to the existing values of the feature. This is the method that will be used here as well (code copied from the same source).
proportions = data['wealth1'].value_counts(normalize=True)
proportions

9.0    0.149665
8.0    0.134037
7.0    0.122297
6.0    0.114937
5.0    0.104183
4.0    0.094909
3.0    0.083603
2.0    0.080604
1.0    0.068153
0.0    0.047612
Name: wealth1, dtype: float64

In [52]:
counts = data['wealth1'].value_counts(normalize=True)
data['wealth1'].fillna(pd.Series(np.random.choice(counts.index, size=len(data.index), p=counts)), inplace=True)
data['wealth1'].value_counts()

9.0    14276
8.0    12829
7.0    11630
6.0    11018
5.0     9933
4.0     9023
3.0     7857
2.0     7683
1.0     6603
0.0     4560
Name: wealth1, dtype: int64

- adi

In [53]:
data.adi.dtypes

dtype('float64')

In [54]:
data.adi.isna().sum()

132

In [55]:
data['adi'].loc[data['adi'] == " "].sum()

0.0

In [56]:
data.adi.describe()

count    95280.000000
mean       187.356402
std        137.019184
min          0.000000
25%         65.000000
50%        175.000000
75%        279.000000
max        651.000000
Name: adi, dtype: float64

The feature is continuous and bears 132 NaNs.

- dma

In [57]:
data.dma.nunique()

206

In [58]:
data.dma.isna().sum()

132

In [59]:
data['dma'].loc[data['dma'] == " "].sum()

0.0

In [60]:
data.dma.describe()

count    95280.000000
mean       664.004072
std        116.363600
min          0.000000
25%        561.000000
50%        635.000000
75%        801.000000
max        881.000000
Name: dma, dtype: float64

As with adi, this feature has exactly 132 NaNs.

- msa

In [61]:
data.msa.describe()

count    95280.000000
mean      3527.744102
std       2863.904737
min          0.000000
25%        520.000000
50%       3350.000000
75%       5960.000000
max       9360.000000
Name: msa, dtype: float64

In [62]:
data.msa.isna().sum()

132

In [63]:
data['msa'].loc[data['msa'] == " "].sum()

0.0

As with adi and msa, this feature has exactly 132 NaNs.

In [64]:
na_rows = data[data['adi'].isna()]
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 150)
na_rows

,odatedw,osource,tcode,state,zip,mailcode,pvastate,dob,noexch,recinhse,...,target_d,hphone_d,rfa_2r,rfa_2f,rfa_2a,mdmaud_r,mdmaud_f,mdmaud_a,cluster2,geocode2
577,8601,BHG,2,FL,33756,,,708,0,,...,0.0,1,L,1,E,X,X,X,NaN,A
1119,8601,SPN,0,FL,34642,,,0,0,,...,0.0,0,L,1,F,X,X,X,NaN,A
2250,8801,DNA,1,GA,31535,,,6201,0,,...,0.0,1,L,1,G,X,X,X,NaN,A
3326,9501,SYN,0,GA,31217,,,0,0,,...,0.0,0,L,3,G,X,X,X,NaN,A
5558,9001,SSS,1,NC,28625,,,2101,0,,...,6.0,1,L,4,E,X,X,X,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90627,9401,L01,1,WA,98375,,,4801,0,,...,0.0,0,L,1,F,X,X,X,NaN,A
90993,9501,LIF,28,SC,59887-,,,2501,0,,...,0.0,0,L,1,F,X,X,X,NaN,A
92870,8601,BHG,0,FL,34624,,,606,0,,...,0.0,1,L,4,E,X,X,X,NaN,A
93624,9101,HAN,2,NC,28370,,,1101,0,,...,0.0,0,L,1,F,X,X,X,NaN,A


adi, dma and msa all have the exact same number of nans and at the exact same locations. In this case and since the number of rows is practically inconsequential in the face of the vastness of data in the dataset, the rows with the nans will be dropped.

In [65]:
data = data[data['adi'].notna()]    # and since all of the features in question have nans at the on the same rows, it is enought to use only one of their names to filter them out
data.shape

(95280, 456)

In [66]:
data.to_csv('learningSet_ml.csv')